In [1]:
from Executor import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [31]:
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("augment").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/segmented/").\
    build()

Pretrained Vgg16 model loaded.
Found 6166 images belonging to 3 classes.
Found 1461 images belonging to 3 classes.
initialized training data from: ../data/segmented/train
initialized validation data from: ../data/segmented/valid
found number of softmax classes: 3


In [3]:
fc_model = executor.get_rescaled_fc_model(new_dropout=0.5)

getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)


In [4]:
# load a finely tuned model
fc_model.load_weights("fc_model_val_loss=0_71.segmented.h5")

In [7]:
# load the precomputed conv. model outputs
executor.load_precomputed_conv_models()

loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7f30f0a5e518>

In [8]:
executor.load_model_from_file("vgg16_with_dense_layer_trained_to_0_71.segmented.h5")

<Executor.Executor instance at 0x7f30f0a5e518>

In [13]:
fc_model = executor.get_rescaled_fc_model(new_dropout=0.0)

getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.0, ' to: ', 0.0)


In [32]:
fc_model.save_weights("overfitted_fc_model.dropout_0.h5")

In [14]:
opt=Adam(lr=0.000001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])    

In [15]:
# Build extremely overfitted model with a 0.0 dropout
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 7s - loss: 0.1513 - acc: 0.9854 - val_loss: 0.7373 - val_acc: 0.6879
Epoch 2/5
6166/6166 [==============================] - 6s - loss: 0.1251 - acc: 0.9904 - val_loss: 0.7426 - val_acc: 0.6886
Epoch 3/5
6166/6166 [==============================] - 6s - loss: 0.1074 - acc: 0.9934 - val_loss: 0.7447 - val_acc: 0.6879
Epoch 4/5
6166/6166 [==============================] - 6s - loss: 0.0924 - acc: 0.9964 - val_loss: 0.7470 - val_acc: 0.6899
Epoch 5/5
6166/6166 [==============================] - 6s - loss: 0.0798 - acc: 0.9976 - val_loss: 0.7583 - val_acc: 0.6913


In [38]:
gen = image.ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None)

In [33]:
conv_model = executor.conv_model

In [34]:
for layer in conv_model.layers:
    layer.trainable = False
for layer in fc_model.layers:
    conv_model.add(layer)    

In [35]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
print executor.train_path
print executor.val_batches

../data/segmented/train


In [39]:
aug_batches = executor.vgg.get_batches(path=executor.train_path, gen=gen, batch_size=executor.batch_size)

Found 6166 images belonging to 3 classes.


In [40]:
conv_model.fit_generator(aug_batches, samples_per_epoch=aug_batches.nb_sample, validation_data=executor.val_batches, 
                         nb_val_samples=executor.val_batches.nb_sample, nb_epoch=5)

Epoch 1/5
6166/6166 [==============================] - 199s - loss: 1.1554 - acc: 0.5527 - val_loss: 0.7583 - val_acc: 0.6913
Epoch 2/5
6166/6166 [==============================] - 199s - loss: 1.1622 - acc: 0.5551 - val_loss: 0.7436 - val_acc: 0.7036
Epoch 3/5
6166/6166 [==============================] - 199s - loss: 1.1603 - acc: 0.5435 - val_loss: 0.7427 - val_acc: 0.7009
Epoch 4/5
6166/6166 [==============================] - 199s - loss: 1.1602 - acc: 0.5517 - val_loss: 0.7561 - val_acc: 0.6899
Epoch 5/5
6166/6166 [==============================] - 199s - loss: 1.1661 - acc: 0.5493 - val_loss: 0.7591 - val_acc: 0.6920


In [41]:
opt=Adam(lr=0.00001)
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])    
conv_model.fit_generator(aug_batches, samples_per_epoch=aug_batches.nb_sample, validation_data=executor.val_batches, 
                         nb_val_samples=executor.val_batches.nb_sample, nb_epoch=5)

Epoch 1/5
6166/6166 [==============================] - 199s - loss: 1.1468 - acc: 0.5553 - val_loss: 0.7562 - val_acc: 0.6934
Epoch 2/5
6166/6166 [==============================] - 198s - loss: 1.1387 - acc: 0.5560 - val_loss: 0.7583 - val_acc: 0.6913
Epoch 3/5
6166/6166 [==============================] - 198s - loss: 1.1689 - acc: 0.5467 - val_loss: 0.7555 - val_acc: 0.6899
Epoch 4/5
6166/6166 [==============================] - 198s - loss: 1.1422 - acc: 0.5511 - val_loss: 0.7641 - val_acc: 0.6940
Epoch 5/5
6166/6166 [==============================] - 199s - loss: 1.1749 - acc: 0.5376 - val_loss: 0.7757 - val_acc: 0.6817


In [42]:
conv_model.save_weights("conv_model.data_augmented.segmented.weights.h5")

In [43]:
executor = ExecutorBuilder().\
    with_runID("augment").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/segmented/").\
    build()

Pretrained Vgg16 model loaded.
Found 6166 images belonging to 3 classes.
Found 1461 images belonging to 3 classes.
initialized training data from: ../data/segmented/train
initialized validation data from: ../data/segmented/valid
found number of softmax classes: 3


In [44]:
executor.load_model_from_file("conv_model.data_augmented.segmented.weights.h5")

<Executor.Executor instance at 0x7f30183650e0>

In [45]:
executor.build_predictions_on_test_data()

Found 512 images belonging to 1 classes.


<Executor.Executor instance at 0x7f30183650e0>

In [46]:
executor.save_predictions_to_file("predictions.vgg16_data_aug.loss_77.h5")

/home/ubuntu/kaggle_cervical_cancer_code/src/predictions.vgg16_data_aug.loss_77.h5